# <div style="text-align: right"> Chapter __11__</div>

# __Strings with stringr__

In [2]:
# libraries
library(tidyverse)

# config
repr_html.tbl_df <- function(obj, ..., rows = 6) repr:::repr_html.data.frame(obj, ..., rows = rows)
options(dplyr.summarise.inform = FALSE)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.1
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## __String Length__

In [3]:
str_length(c('a', 'R for data science', NA))

[1]  1 18 NA

### __Combining Strings__

In [4]:
str_c('x', 'y')

[1] "xy"

In [6]:
str_c('x', 'y', sep = ', ')

[1] "x, y"

Like most other functions in R, missing values are contagious. If you
want them to print as "NA" , use `str_replace_na()` :

In [7]:
x <- c('abc', NA)
str_c('|-', x, '-|')

[1] "|-abc-|" NA

In [8]:
str_c('|-', str_replace_na(x), '-|')

[1] "|-abc-|" "|-NA-|"

As shown in the preceding code, `str_c()` is vectorized, and it auto‐
matically recycles shorter vectors to the same length as the longest:

In [9]:
str_c('prefix-', c('a', 'b', 'c'), '-suffix')

[1] "prefix-a-suffix" "prefix-b-suffix" "prefix-c-suffix"

### __Subsetting Strings__

You can extract parts of a string using `str_sub()` . As well as the
string, `str_sub()` takes start and end arguments that give the
(inclusive) position of the substring:

In [11]:
x <- c('Apple', 'Banana', 'Pear')
str_sub(x, 1, 3)

[1] "App" "Ban" "Pea"

In [12]:
str_sub(x, -3, -1)

[1] "ple" "ana" "ear"

Note that `str_sub()` won’t fail if the string is too short; it will just
return as much as possible:

In [13]:
str_sub('ab', 1, 5)

[1] "ab"

You can also use the assignment form of `str_sub()` to modify
strings:

In [15]:
str_sub(x, 1, 1) <- str_to_lower(str_sub(x, 1, 1))
(x)

[1] "apple"  "banana" "pear"



What does `str_trim()` do? What’s the opposite of `str_trim()`?


In [28]:
?str_trim

str_trim {stringr},R Documentation
string,A character vector.
side,"Side on which to remove whitespace (left, right or both)."


In [29]:
str_trim('  12345  ')

[1] "12345"

In [30]:
str_trim('  12345  ', side = 'left')

[1] "12345  "

In [34]:
str_pad('12345', 9)

[1] "    12345"

In [35]:
str_pad('123', 5, side = 'both')

[1] " 123 "

Write a function that turns (e.g.) a vector `c("a", "b", "c")`
into the string a, b, and c . Think carefully about what it
should do if given a vector of length 0, 1, or 2.

In [85]:
str_sep_comma <- function(vector) {
    if (length(vector) == 0) {
        return(vector)
    }
    else if (length(vector) == 1) {
        return(vector)
    }
    else if (length(vector) == 2) {
        return(str_c(vector[1], ' and ', vector[2]))
    }
    else {
        head <- str_c(vector[seq_len(length(vector) - 1)], ',')
        tail <- str_c("and", vector[[length(vector)]], sep = " ")
        return(str_c(c(head, tail), collapse = " "))
    }
}

In [86]:
str_sep_comma(c('a', 'c', 'e', 'f', 't', 'd'))

[1] "a, c, e, f, t, and d"

## __Matching Patterns with Regular Expressions__

Regexps are a very terse language that allow you to describe patterns
in strings. They take a little while to get your head around, but once
you understand them, you’ll find them extremely useful.
To learn regular expressions, we’ll use `str_view()` and
`str_view_all()` . These functions take a character vector and a regular expression, and show you how they match. We’ll start with very
simple regular expressions and then gradually get more and more
complicated. Once you’ve mastered pattern matching, you’ll learn
how to apply those ideas with various stringr functions.

In [18]:
x <- c('apple', 'banana', 'pear')
str_view(x, 'an')

HTML widgets cannot be represented in plain text (need html)

The next step up in complexity is `.` , which matches any character
(except a newline):

In [19]:
str_view(x, '.a.')

HTML widgets cannot be represented in plain text (need html)

But if `"."` matches any character, how do you match the character
`"."` ? You need to use an “escape” to tell the regular expression you
want to match it exactly, not use its special behavior. Like strings,
regexps use the backslash, `\` , to escape special behavior. So to match
an `.` , you need the regexp `\.` . Unfortunately this creates a problem.
We use strings to represent regular expressions, and `\` is also used as
an escape symbol in strings. So to create the regular expression `\`.
we need the string `"\\."` :

In [21]:
str_view(c('abc', 'a.c', 'bef'), 'a\\.c')

HTML widgets cannot be represented in plain text (need html)

If `\` is used as an escape character in regular expressions, how do
you match a literal `\` ? Well you need to escape it, creating the regular
expression `\\` . To create that regular expression, you need to use a
string, which also needs to escape `\` . That means to match a literal `\`
you need to write `"\\\\"` —you need four backslashes to match one!

In [22]:
x <- 'a\\b'
writeLines(x)

a\b


In [23]:
str_view(x, '\\\\')

HTML widgets cannot be represented in plain text (need html)

### __Anchors__

By default, regular expressions will match any part of a string. It’s
often useful to anchor the regular expression so that it matches from
the start or end of the string. You can use:

* `^` to match the start of the string.
* `$` to match the end of the string.

In [24]:
x <- c('apple', 'banana', 'pear')
str_view(x, '^a')

HTML widgets cannot be represented in plain text (need html)

In [25]:
str_view(x, 'a$')

HTML widgets cannot be represented in plain text (need html)

To force a regular expression to only match a complete string,
anchor it with both `^` and `$` :

In [26]:
x <- c('apple pie', 'apple', 'apple cake')
str_view(x, 'apple')

HTML widgets cannot be represented in plain text (need html)

In [27]:
str_view(x, '^apple$')

HTML widgets cannot be represented in plain text (need html)

## __Character Classes and Alternatives__

There are a number of special patterns that match more than one
character. You’ve already seen . , which matches any character apart
from a newline. There are four other useful tools:

* `\d` matches any digit.
* `\s` matches any whitespace (e.g., space, tab, newline).
* `[abc]` matches a, b, or c.
* `[^abc]` matches anything except a, b, or c.

You can use alternation to pick between one or more alternative pat‐
terns. For example, `abc|d..f` will match either "abc" , or "deaf" .
Note that the precedence for `|` is low, so that `abc|xyz` matches abc
or xyz not abcyz or abxyz . Like with mathematical expressions, if
precedence ever gets confusing, use parentheses to make it clear
what you want:

In [87]:
str_view(c('grey', 'gray'), 'gr(e|a)y')

HTML widgets cannot be represented in plain text (need html)

Create regular expressions to find all words that:
* Start with a vowel.
* Only contain consonants. (Hint: think about matching
“not”-vowels.)
* End with ed , but not with eed .
* End with ing or ize .

In [95]:
x <- c('hola', 'adios', 'queso', 'amor amor', 'ether', 'thneed', 'studied')
z <- c('engineering', 'eating', 'cheese', 'categorize', 'hipnotize', 'hello')

In [96]:
str_view(x, '^[aeiou]')

HTML widgets cannot be represented in plain text (need html)

In [97]:
str_view(x, '^[^(aeiou)]')

HTML widgets cannot be represented in plain text (need html)

In [105]:
str_view(x, '[^e]ed$')

HTML widgets cannot be represented in plain text (need html)

In [106]:
str_view(z, '(ing|ize)$')

HTML widgets cannot be represented in plain text (need html)

### __Repetition__

The next step up in power involves controlling how many times a
pattern matches:

* `?`:0 or 1
* `+`: 1 or more
* `*`: 0 or more

In [107]:
x <- "1888 is the longest year in Roman numerals: MDCCCLXXXVIII"
str_view(x, 'CC?')

HTML widgets cannot be represented in plain text (need html)

In [108]:
str_view(x, 'CC+')

HTML widgets cannot be represented in plain text (need html)

In [109]:
str_view(x, 'C[LX]+')

HTML widgets cannot be represented in plain text (need html)

In [110]:
str_view(x, '[CLX]+')

HTML widgets cannot be represented in plain text (need html)

You can also specify the number of matches precisely:

* `{n}`: exactly n
* `{n,}`: n or more
* `{,m}`: at most m
* `{n,m}`: between n and m

In [111]:
str_view(x, 'C{2}')

HTML widgets cannot be represented in plain text (need html)

In [112]:
str_view(x, 'C{2,}')

HTML widgets cannot be represented in plain text (need html)

In [113]:
str_view(x, 'C{2,3}')

HTML widgets cannot be represented in plain text (need html)

By default these matches are “greedy”: they will match the longest
string possible. You can make them “lazy,” matching the shortest
string possible, by putting a ? after them. This is an advanced fea‐
ture of regular expressions, but it’s useful to know that it exists:

In [114]:
str_view(x, 'C{2,3}?')

HTML widgets cannot be represented in plain text (need html)

In [115]:
str_view(x, 'C[LX]+?')

HTML widgets cannot be represented in plain text (need html)

Write a regular expression that matches a word if it’s probably written in British English, not American English.

In the general case, this is hard, and could require a dictionary. But, there are a few heuristics to consider that would account for some common cases: British English tends to use the following:

* “ou” instead of “o”
* use of “ae” and “oe” instead of “a” and “o”
* ends in ise instead of ize
* ends in yse


## __Grouping and Backreferences__

Earlier, you learned about parentheses as a way to disambiguate
complex expressions. They also define “groups” that you can refer to
with backreferences, like \1 , \2 , etc. For example, the following regular expression finds all fruits that have a repeated pair of letters:

In [116]:
str_view(fruit, '(..)\\1', match = TRUE)

HTML widgets cannot be represented in plain text (need html)

2. Construct regular expressions to match words that:

* Start and end with the same character.
* Contain a repeated pair of letters (e.g., “church” contains “ch”
repeated twice).
* Contain one letter repeated in at least three places (e.g.,
“eleven” contains three “e”s).

In [128]:
x <- c('aura', 'ethernet', 'meem', 'strings', 'america', 'example', 'ejempli')
str_view(x, '^(.)(.*)(\\1)$')

HTML widgets cannot be represented in plain text (need html)

In [134]:
str_view(fruit, '([A-Za-z][A-Za-z]).*\\1', match = TRUE)

HTML widgets cannot be represented in plain text (need html)

## __Detect Matches__

To determine if a character vector matches a pattern, use
`str_detect()` . It returns a logical vector the same length as the
input:

In [135]:
x <- c('apple', 'banana', 'pear')

In [136]:
str_detect(x, 'e')

[1]  TRUE FALSE  TRUE

Remember that when you use a logical vector in a numeric context,
`FALSE` becomes `0` and `TRUE` becomes `1`. That makes `sum()` and
`mean()` useful if you want to answer questions about matches across
a larger vector:

In [137]:
# how many common words start with t?
sum(str_detect(words, '^t'))

[1] 65

In [138]:
# what proportion of common words end with a vowel?
mean(str_detect(words, '[aeiou]$'))

[1] 0.2765306

In [139]:
mean(str_detect(words, '[^(aeiou)]$'))

[1] 0.7234694

When you have complex logical conditions (e.g., match a or b but
not c unless d) it’s often easier to combine multiple `str_detect()`
calls with logical operators, rather than trying to create a single reg‐
ular expression. For example, here are two ways to find all words
that don’t contain any vowels:

In [140]:
# find all words containing at least one vowel,
# and negate
no_vowels_1 <- !str_detect(words, '[aeiou]')
# find all words consisting only of consonants (non-vowels)
no_vowels_2 <- str_detect(words, '^[^aeiou]+$')

identical(no_vowels_1, no_vowels_2)

[1] TRUE

The results are identical, but I think the first approach is signifi‐
cantly easier to understand. If your regular expression gets overly
complicated, try breaking it up into smaller pieces, giving each piece
a name, and then combining the pieces with logical operations.

A common use of `str_detect()` is to select the elements that match
a pattern. You can do this with logical subsetting, or the convenient
`str_subset()` wrapper:

In [143]:
words[str_detect(words, 'x$')]

[1] "box" "sex" "six" "tax"

In [144]:
str_subset(words, 'x$')

[1] "box" "sex" "six" "tax"

Typically, however, your strings will be one column of a data frame,
and you’ll want to use `filter` instead:

In [145]:
df <- tibble(
    word = words,
    i = seq_along(word)
)

In [146]:
df %>%
    filter(str_detect(words, 'x$'))

word,i
<chr>,<int>
box,108
sex,747
six,772
tax,841


A variation on `str_detect()` is `str_count()` : rather than a simple
yes or no, it tells you how many matches there are in a string:

In [147]:
x <- c('apple', 'banana', 'pear')
str_count(x, 'a')

[1] 1 3 1

In [149]:
# on average, how many vowels per word?
mean(str_count(words, '[aeiou]'))

[1] 1.991837

In [150]:
# its natural to use str_count() with mutate()
df %>%
    mutate(vowels = str_count(word, '[aeiou]'),
           consonants = str_count(word, '[^aeiou]'))

word,i,vowels,consonants
<chr>,<int>,<int>,<int>
a,1,1,0
able,2,2,2
about,3,3,2
⋮,⋮,⋮,⋮
yet,978,1,2
you,979,2,1
young,980,2,3


Note that matches never overlap. For example, in "abababa" , how
many times will the pattern "aba" match? Regular expressions say
two, not three:

In [151]:
str_count('abababa', 'aba')

[1] 2

In [152]:
str_view_all('abababa', 'aba')

HTML widgets cannot be represented in plain text (need html)

## __Extract Matches__

In [153]:
length(sentences)

[1] 720

In [154]:
head(sentences)

[1] "The birch canoe slid on the smooth planks." 
[2] "Glue the sheet to the dark blue background."
[3] "It's easy to tell the depth of a well."     
[4] "These days a chicken leg is a rare dish."   
[5] "Rice is often served in round bowls."       
[6] "The juice of lemons makes fine punch."

Imagine we want to find all sentences that contain a color. We first
create a vector of color names, and then turn it into a single regular
expression:

In [155]:
colors <- c('red', 'orange', 'yellow', 'green', 'blue', 'purple')
color_match <- str_c(colors, collapse = '|')
color_match

[1] "red|orange|yellow|green|blue|purple"

In [156]:
# now we can select sentences that contain a color,
# and then extract the color to figure out which one it is
has_color <- str_subset(sentences, color_match)
matches <- str_extract(has_color, color_match)

head(matches)

[1] "blue" "blue" "red"  "red"  "red"  "blue"

Note that `str_extract()` only extracts the first match. We can see
that most easily by first selecting all the sentences that have more
than one match

In [157]:
more <- sentences[str_count(sentences, color_match) > 1]
str_view_all(more, color_match)

HTML widgets cannot be represented in plain text (need html)

In [158]:
str_extract(more, color_match)

[1] "blue"   "green"  "orange"

This is a common pattern for stringr functions, because working
with a single match allows you to use much simpler data structures.
To get all matches, use `str_extract_all()` . It returns a list:

In [159]:
str_extract_all(more, color_match)

[[1]]
[1] "blue" "red" 

[[2]]
[1] "green" "red"  

[[3]]
[1] "orange" "red"

If you use `simplify = TRUE` , `str_extract_all()` will return a
matrix with short matches expanded to the same length as the
longest:

In [160]:
str_extract_all(more, color_match, simplify = TRUE)

blue,red
green,red
orange,red


In [161]:
x <- c('a', 'a b', 'a b c')
str_extract_all(x, '[a-z]', simplify = TRUE)

a,,
a,b,
a,b,c


2. From the Harvard sentences data, extract:
* The first word from each sentence.
* All words ending in ing .
* All plurals.

In [162]:
str_extract(sentences, "[A-ZAa-z]+")

[1] "The"        "Glue"       "It"         "These"      "Rice"      
  [6] "The"        "The"        "The"        "Four"       "Large"     
 [11] "The"        "A"          "The"        "Kick"       "Help"      
 [16] "A"          "Smoky"      "The"        "The"        "The"       
 [21] "The"        "The"        "Press"      "The"        "The"       
 [26] "Two"        "Her"        "The"        "It"         "Read"      
 [31] "Hoist"      "Take"       "Note"       "Wipe"       "Mend"      
 [36] "The"        "The"        "The"        "The"        "What"      
 [41] "A"          "The"        "Sickness"   "The"        "The"       
 [46] "Lift"       "The"        "Hop"        "The"        "Mesh"      
 [51] "The"        "The"        "Adding"     "The"        "A"         
 [56] "The"        "March"      "A"          "Place"      "Both"      
 [61] "We"         "Use"        "He"         "The"        "A"         
 [66] "Cars"       "The"        "This"       "The"        "Those"     
 [71] "A"          "The"        "The"        "The"        "The"       
 [76] "A"          "The"        "The"        "The"        "The"       
 [81] "The"        "See"        "There"      "The"        "The"       
 [86] "The"        "Cut"        "Men"        "Always"     "He"        
 [91] "The"        "A"          "A"          "The"        "The"       
 [96] "Bail"       "The"        "A"          "Ten"        "The"       
[101] "Oak"        "Cats"       "The"        "Open"       "Add"       
[106] "Thieves"    "The"        "Act"        "The"        "Move"      
[111] "The"        "Leaves"     "The"        "Split"      "Burn"      
[116] "He"         "Weave"      "Hemp"       "A"          "We"        
[121] "Type"       "The"        "The"        "The"        "Paste"     
[126] "The"        "It"         "The"        "Feel"       "The"       
[131] "A"          "He"         "Pluck"      "Two"        "The"       
[136] "Bring"      "Write"      "Clothes"    "We"         "Port"      
[141] "The"        "Guess"      "A"          "The"        "These"     
[146] "Pure"       "The"        "The"        "Mud"        "The"       
[151] "The"        "A"          "He"         "The"        "The"       
[156] "The"        "The"        "We"         "She"        "The"       
[161] "The"        "At"         "Drop"       "A"          "An"        
[166] "Wood"       "The"        "He"         "A"          "A"         
[171] "Steam"      "The"        "There"      "The"        "Torn"      
[176] "Sunday"     "The"        "The"        "They"       "Add"       
[181] "Acid"       "Fairy"      "Eight"      "The"        "A"         
[186] "Add"        "We"         "There"      "He"         "She"       
[191] "The"        "Corn"       "Where"      "The"        "Sell"      
[196] "The"        "The"        "Bring"      "They"       "Farmers"   
[201] "The"        "The"        "Float"      "A"          "A"         
[206] "The"        "After"      "The"        "He"         "Even"      
[211] "The"        "The"        "The"        "Do"         "Lire"      
[216] "The"        "It"         "Write"      "The"        "The"       
[221] "A"          "Coax"       "Schools"    "The"        "They"      
[226] "The"        "The"        "Jazz"       "Rake"       "Slash"     
[231] "Try"        "They"       "He"         "They"       "The"       
[236] "Whitings"   "Some"       "Jerk"       "A"          "Madam"     
[241] "On"         "The"        "This"       "Add"        "The"       
[246] "The"        "The"        "To"         "The"        "Jump"      
[251] "Yell"       "They"       "Both"       "In"         "The"       
[256] "The"        "Ducks"      "Fruit"      "These"      "Canned"    
[261] "The"        "Carry"      "The"        "We"         "Gray"      
[266] "The"        "High"       "Tea"        "A"          "A"         
[271] "The"        "Find"       "Cut"        "The"        "Look"      
[276] "The"        "Nine"       "The"        "The"        "Soak"      
[281] "The"        "A"    

In [163]:
pattern <- "\\b[A-Za-z]+ing\\b"
sentences_with_ing <- str_detect(sentences, pattern)
unique(unlist(str_extract_all(sentences[sentences_with_ing], pattern))) %>%
  head()

[1] "spring"  "evening" "morning" "winding" "living"  "king"

In [164]:
unique(unlist(str_extract_all(sentences, "\\b[A-Za-z]{3,}s\\b"))) %>%
  head()

[1] "planks" "days"   "bowls"  "lemons" "makes"  "hogs"

## __Grouped Matches__

Earlier in this chapter we talked about the use of parentheses for
clarifying precedence and for backreferences when matching. You
can also use parentheses to extract parts of a complex match. For
example, imagine we want to extract nouns from the sentences. As a
heuristic, we’ll look for any word that comes after “a” or “the”. Defining a “word” in a regular expression is a little tricky, so here I use a simple approximation—a sequence of at least one character that isn’t
a space:

In [165]:
noun <- '(a|the) ([^ ]+)'

In [166]:
has_noun <- sentences %>%
    str_subset(noun) %>%
    head(10)

In [167]:
has_noun %>%
    str_extract(noun)

[1] "the smooth" "the sheet"  "the depth"  "a chicken"  "the parked"
 [6] "the sun"    "the huge"   "the ball"   "the woman"  "a helps"

`str_extract()` gives us the complete match; `str_match()` gives
each individual component. Instead of a character vector, it returns
a matrix, with one column for the complete match followed by one
column for each group:

In [168]:
has_noun %>%
    str_match(noun)

the smooth,the,smooth
the sheet,the,sheet
the depth,the,depth
a chicken,a,chicken
the parked,the,parked
the sun,the,sun
the huge,the,huge
the ball,the,ball
the woman,the,woman
a helps,a,helps


If your data is in a tibble, it’s often easier to use `tidyr::extract()` .
It works like `str_match()` but requires you to name the matches,
which are then placed in new columns:

In [169]:
tibble(sentence = sentences) %>%
    extract(sentence, c('article', 'noun'), '(a|the) ([^ ]+)',
            remove = FALSE)

sentence,article,noun
<chr>,<chr>,<chr>
The birch canoe slid on the smooth planks.,the,smooth
Glue the sheet to the dark blue background.,the,sheet
It's easy to tell the depth of a well.,the,depth
⋮,⋮,⋮
A severe storm tore down the barn.,the,barn.
She called his name many times.,NA,NA
"When you hear the bell, come quickly.",the,"bell,"


## __Replacing Matches__

`str_replace()` and `str_replace_all()` allow you to replace
matches with new strings. The simplest use is to replace a pattern
with a fixed string:

In [170]:
x <- c('apple', 'pear', 'banana')
str_replace(x, '[aeiou]', '-')

[1] "-pple"  "p-ar"   "b-nana"

In [171]:
str_replace_all(x, '[aeiou]', '-')

[1] "-ppl-"  "p--r"   "b-n-n-"

With `str_replace_all()` you can perform multiple replacements
by supplying a named vector:

In [172]:
x <- c('1 house', '2 cars', '3 people')
str_replace_all(x, c('1' = 'one', '2' = 'two', '3' = 'three'))

[1] "one house"    "two cars"     "three people"

Instead of replacing with a fixed string you can use backreferences
to insert components of the match. In the following code, I flip the
order of the second and third words:

In [173]:
sentences %>%
    str_replace("([^ ]+) ([^ ]+) ([^ ]+)", "\\1 \\3 \\2") %>%
    head(5)

[1] "The canoe birch slid on the smooth planks." 
[2] "Glue sheet the to the dark blue background."
[3] "It's to easy tell the depth of a well."     
[4] "These a days chicken leg is a rare dish."   
[5] "Rice often is served in round bowls."

## __Splitting__

Use `str_split()` to split a string up into pieces. For example, we
could split sentences into words:

In [174]:
sentences %>%
    head(5) %>%
    str_split(' ')

[[1]]
[1] "The"     "birch"   "canoe"   "slid"    "on"      "the"     "smooth" 
[8] "planks."

[[2]]
[1] "Glue"        "the"         "sheet"       "to"          "the"        
[6] "dark"        "blue"        "background."

[[3]]
[1] "It's"  "easy"  "to"    "tell"  "the"   "depth" "of"    "a"     "well."

[[4]]
[1] "These"   "days"    "a"       "chicken" "leg"     "is"      "a"      
[8] "rare"    "dish."  

[[5]]
[1] "Rice"   "is"     "often"  "served" "in"     "round"  "bowls."

Because each component might contain a different number of
pieces, this returns a list. If you’re working with a length-1 vector,
the easiest thing is to just extract the first element of the list:

In [175]:
'a|b|c|d' %>%
    str_split('\\|') %>%
    .[[1]]

[1] "a" "b" "c" "d"

In [176]:
# or return a matrix
sentences %>%
    head(5) %>%
    str_split(' ', simplify = TRUE)

The,birch,canoe,slid,on,the,smooth,planks.,
Glue,the,sheet,to,the,dark,blue,background.,
It's,easy,to,tell,the,depth,of,a,well.
These,days,a,chicken,leg,is,a,rare,dish.
Rice,is,often,served,in,round,bowls.,,


In [177]:
# request max number of pieaces
fields <- c("Name: Hadley", "Country: NZ", "Age: 35")
fields %>%
    str_split(': ', n = 2, simplify = TRUE)

Name,Hadley
Country,NZ
Age,35


Instead of splitting up strings by patterns, you can also split up by
character, line, sentence, and word `boundary()` s:

In [178]:
x <- "This is a sentence. This is another sentence."
str_view_all(x, boundary('word'))

HTML widgets cannot be represented in plain text (need html)

In [179]:
str_split(x, ' ')[[1]]

[1] "This"      "is"        "a"         "sentence." "This"      "is"       
[7] "another"   "sentence."

In [180]:
str_split(x, boundary('word'))[[1]]

[1] "This"     "is"       "a"        "sentence" "This"     "is"       "another" 
[8] "sentence"

## __Find Matches__
`str_locate()` and `str_locate_all()` give you the starting and
ending positions of each match. These are particularly useful when
none of the other functions does exactly what you want. You can use
`str_locate()` to find the matching pattern, and `str_sub()` to
extract and/or modify them.

## __Other Types of Pattern__

When you use a pattern that’s a string, it’s automatically wrapped
into a call to `regex()` :

In [181]:
# the regular call
str_view(fruit, 'nana')

HTML widgets cannot be represented in plain text (need html)

In [183]:
# is shorthand for
str_view(fruit, regex('nana'))

HTML widgets cannot be represented in plain text (need html)

`ignore_case = TRUE` allows characters to match either their
uppercase or lowercase forms. This always uses the current
locale:

In [184]:
bananas <- c('banana', 'Banana', 'BANANA')
str_view(bananas, 'banana')

HTML widgets cannot be represented in plain text (need html)

In [185]:
str_view(bananas, regex('banana', ignore_case = TRUE))

HTML widgets cannot be represented in plain text (need html)

`multiline = TRUE` allows `^` and `$` to match the start and end of
each line rather than the start and end of the complete string:

In [186]:
x <- "Line 1\nLine 2\nLine 3"

In [187]:
str_extract_all(x, "^Line")[[1]]

[1] "Line"

In [189]:
str_extract_all(x, regex("^Line", multiline = TRUE))[[1]]

[1] "Line" "Line" "Line"

`comments = TRUE` allows you to use comments and white space
to make complex regular expressions more understandable.
Spaces are ignored, as is everything after # . To match a literal
space, you’ll need to escape it: `"\\ "` .

In [190]:
phone <- regex('
    \\(?     # optional opening parens
    (\\d{3}) # area code
    [)- ]?   # optional closing parens, dash, or space
    (\\d{3}) # another three numbers
    [ -]?    # optional space or dash
    (\\d{3}) # three more numbers
', comments = TRUE)

In [191]:
str_match('514-791-8141', phone)

514-791-814,514,791,814


`dotall = TRUE` allows . to match everything, including `\n` .